<a href="https://colab.research.google.com/github/SandeepJosyula/NeuralNetworks/blob/master/SJ_ComputerVision_Project_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CV Project -Face Mask Segmentation

**Project Description:**

In this hands-on project, the goal is to build a Face Mask Segmentation model
which includes building a face detector to locate the position of a face in an
image.

**Data Description:** WIDER Face Dataset

WIDER FACE dataset is a Face Mask Segmentation benchmark dataset, of which
images are selected from the publicly available WIDER dataset. This data have 32,203 images and 393,703 faces are labeled with a high degree of
variability in scale, pose and occlusion as depicted in the sample images.
In this project, we are using 409 images and around 1000 faces for ease of
computation.

We will be using transfer learning on an already trained model to build our
segmenter. We will perform transfer learning on the MobileNet model which is
already trained to perform image segmentation. We will need to train the last 6-7 layers and freeze the remaining layers to train the model for face mask
segmentation. To be able to train the MobileNet model for face mask
segmentation, we will be using the WIDER FACE dataset for various images with a
single face and multiple faces. The output of the model is the face mask
segmented data which masks the face in an image. We learn to build a face mask
segmentation model using Keras supported by Tensorflow.

**Reference**
Acknowledgment for the datasets.
http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/

Mobile Net paper: https://arxiv.org/pdf/1704.04861.pdf

**Objective:**
In this problem, we use "Transfer Learning" of an Image Segmentation model to
detect any object according to the problem in hand.

Here, we are particularly interested in segmenting faces in a given image
Steps and Tasks:
 - Load the dataset given in form .npy format.
    - We have already extracted the images from wider face-dataset and
added it in the file images.npy. You can directly use this file for this
project.
    - “images.npy” contains details about the image and it’s masks, there
is no separate CSV file for that
    - There is no separate train and test data given
 -  Create Features(images) and labels(mask) using that data.
 - Load the pre-trained model and weights. 
 - Create a model using the above model.
 - Define the Dice Coefficient and Loss function.
 - Compile and fit the model.
 - Evaluate the model.

**Note:** Detailed Instructions for all the above steps are given in the notebook

## Import Libraries

In [28]:
%tensorflow_version 2.x
import tensorflow as tf
tensorflow.__version__

'2.3.0'

In [26]:
import numpy as np

# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [27]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Load

In [29]:
with np.load('/content/drive/My Drive/Study/AI_ML_Study/GreatLearning_UT_Texas/9.Computer_Vision/Project/images.npy', allow_pickle=True) as data:
  ex = data["X"]

AttributeError: ignored

In [30]:
data = np.load('/content/drive/My Drive/Study/AI_ML_Study/GreatLearning_UT_Texas/9.Computer_Vision/Project/images.npy', allow_pickle=True)

In [31]:
data.shape

(409, 2)

In [32]:
data[1]

array([array([[[207, 216, 227, 255],
        [206, 216, 227, 255],
        [207, 216, 227, 255],
        ...,
        [ 35,  33,  34, 255],
        [ 35,  33,  34, 255],
        [ 35,  33,  34, 255]],

       [[207, 216, 227, 255],
        [207, 216, 227, 255],
        [207, 216, 227, 255],
        ...,
        [ 35,  32,  33, 255],
        [ 35,  33,  34, 255],
        [ 35,  33,  34, 255]],

       [[207, 216, 227, 255],
        [207, 216, 227, 255],
        [207, 215, 227, 255],
        ...,
        [ 35,  33,  33, 255],
        [ 35,  33,  34, 255],
        [ 35,  33,  34, 255]],

       ...,

       [[ 31,  21,  17, 255],
        [ 31,  22,  18, 255],
        [ 31,  22,  18, 255],
        ...,
        [  0,   1,   4, 255],
        [  0,   1,   4, 255],
        [  0,   1,   4, 255]],

       [[ 31,  22,  18, 255],
        [ 31,  22,  18, 255],
        [ 31,  22,  18, 255],
        ...,
        [  0,   1,   4, 255],
        [  0,   1,   4, 255],
        [  0,   1,   4, 255]],

     

In [22]:
d1 = data.["array"]

IndexError: ignored

## Data Pre-processing

In [20]:
with np.load('/content/drive/My Drive/Study/AI_ML_Study/GreatLearning_UT_Texas/9.Computer_Vision/Project/images.npy', allow_pickle=True) as data:
  features = data["features"]
  labels = data["label"]
 # notes = data["notes"]
 # points = data["points"]
 # imageWidth = data["imageWidth"] 
 # imageHeight = data["imageHeight"]


AttributeError: ignored

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [ ]:
pickle.Unpickler()

## Load pre-trained model and weights

## Create a new model using the pre-available model

## Define the Dice Coefficient and Loss function

## Compile and fit the model

## Evaluate the model